In [16]:
import pandas as pd
import datetime

from sklearn.linear_model import Lasso

In [17]:
df1 = pd.read_csv("data/sample/sampleAccepted.csv")
df1 = df1.drop(columns=["member_id", "url"])
df1.head().T

,0,1,2,3,4
id,145181203,102630556,135925949,59212770,74554116
loan_amnt,9100,8000,25000,7500,12000
funded_amnt,9100,8000,25000,7500,12000
funded_amnt_inv,9100,8000,25000,7500,12000
term,36 months,36 months,36 months,36 months,36 months
...,...,...,...,...,...
settlement_status,NaN,NaN,NaN,NaN,NaN
settlement_date,NaN,NaN,NaN,NaN,NaN
settlement_amount,NaN,NaN,NaN,NaN,NaN
settlement_percentage,NaN,NaN,NaN,NaN,NaN


In [18]:
def formatTerm(line):
    line = line.strip()
    line = line.split(" ")

    if line[-1] == "months":
        line = int(line[0]) * 30
    
    else:
        raise NotImplementedError(f"{line[-1]} não implementado ainda")

    return line

df1["term"] = df1["term"].apply(formatTerm)
df1.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,145181203,9100.0,9100.0,9100.0,1080,26.31,368.15,E,E4,Rural carrier,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,102630556,8000.0,8000.0,8000.0,1080,16.99,285.19,D,D1,Gas Turbine Mechanic,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,135925949,25000.0,25000.0,25000.0,1080,8.08,784.34,A,A5,HR Business Partner,...,NaN,NaN,DirectPay,N,NaN,NaN,NaN,NaN,NaN,NaN
3,59212770,7500.0,7500.0,7500.0,1080,7.89,234.65,A,A5,manager,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,74554116,12000.0,12000.0,12000.0,1080,8.39,378.20,B,B1,Pharmacy Technician,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df1.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 149 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   id                                          10000 non-null  int64  
 1   loan_amnt                                   10000 non-null  float64
 2   funded_amnt                                 10000 non-null  float64
 3   funded_amnt_inv                             10000 non-null  float64
 4   term                                        10000 non-null  int64  
 5   int_rate                                    10000 non-null  float64
 6   installment                                 10000 non-null  float64
 7   grade                                       10000 non-null  object 
 8   sub_grade                                   10000 non-null  object 
 9   emp_title                                   9257 non-null   object 
 10  emp_length

In [20]:
df1.iloc[:5, 10:20]

,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,desc,purpose,title
0,3 years,OWN,62000.0,Verified,Dec-2018,Late (31-120 days),n,NaN,other,Other
1,8 years,RENT,69000.0,Verified,Mar-2017,Charged Off,n,NaN,debt_consolidation,Debt consolidation
2,2 years,RENT,102000.0,Verified,Jul-2018,Current,n,NaN,credit_card,Credit card refinancing
3,10+ years,MORTGAGE,33000.0,Not Verified,Sep-2015,Fully Paid,n,NaN,debt_consolidation,Debt consolidation
4,10+ years,RENT,32201.0,Not Verified,Mar-2016,Fully Paid,n,NaN,debt_consolidation,Debt consolidation


In [21]:
target = df1["grade"]
target.unique()

array(['E', 'D', 'A', 'B', 'C', 'F', 'G'], dtype=object)

In [22]:
df1 = df1.join(pd.get_dummies(df1["home_ownership"]))
df1 = df1.drop(columns=["home_ownership", "OWN"])
df1

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,ANY,MORTGAGE,NONE,OTHER,RENT
0,145181203,9100.0,9100.0,9100.0,1080,26.31,368.15,E,E4,Rural carrier,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
1,102630556,8000.0,8000.0,8000.0,1080,16.99,285.19,D,D1,Gas Turbine Mechanic,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
2,135925949,25000.0,25000.0,25000.0,1080,8.08,784.34,A,A5,HR Business Partner,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
3,59212770,7500.0,7500.0,7500.0,1080,7.89,234.65,A,A5,manager,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0
4,74554116,12000.0,12000.0,12000.0,1080,8.39,378.20,B,B1,Pharmacy Technician,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,137815471,10000.0,10000.0,10000.0,1080,23.40,389.19,E,E1,Service Manager,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1
9996,5524600,9500.0,9500.0,9500.0,1080,15.80,333.06,C,C3,BLUWORLD,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0
9997,7677306,6000.0,6000.0,6000.0,1080,9.99,193.58,B,B1,supervisor,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0
9998,59251152,6125.0,6125.0,6125.0,1080,16.99,218.35,D,D3,Porter,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1


In [23]:
df1.columns

Index(['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title',
       ...
       'settlement_status', 'settlement_date', 'settlement_amount',
       'settlement_percentage', 'settlement_term', 'ANY', 'MORTGAGE', 'NONE',
       'OTHER', 'RENT'],
      dtype='object', length=153)